In [ ]:
# Function (tools) calling

In [7]:
import os
import sys

project_root = '/mnt/c/Users/a884470/prj/genai-blueprint-main'  # Change this if needed
sys.path.append(os.path.join(project_root, 'python'))


In [8]:
from devtools import debug
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)
#!export PYTHONPATH=":./python"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [9]:
from devtools import debug
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool

from ai_core.llm import get_llm
from ai_core.prompts import def_prompt

### Step 1 : Ask LLM to return an instantiated function  

In [10]:
# llm = get_llm("mistral_large_edenai")
# llm = get_llm("gpt_4omini_edenai")
# llm = get_llm("gpt_35_azure")
llm = get_llm("gpt_4_azure")
#llm = get_llm("llama32_3_ollama")
messages = []


@tool
def add(a: int, b: int) -> int:
    """Addition 2 integer numbers a and b.

    Args:
        a: first int
        b: second int
    """
    print("'add' tool called")
    return a + b


llm_with_tools = llm.bind_tools([add]) # tool_choice="any"
prompt = def_prompt(
    "Use the provided functions  to compute what is 45 + 12; execute the function. Return 'I don't know' if there are not relevant function  "
)
chain = prompt | llm_with_tools
ai_msg = chain.invoke({})

debug(ai_msg)

2024-11-14 09:55:39.692 | INFO     | config:yaml_file_config:43 - load /mnt/c/Users/a884470/prj/genai-blueprint-main/app_conf.yaml


2024-11-14 09:55:39.742 | INFO     | config:yaml_file_config:52 - Override config from env. variable: azure
2024-11-14 09:55:39.775 | INFO     | ai_core.llm:get_llm:409 - get LLM:'gpt_4_azure'


/tmp/ipykernel_5275/1057213440.py:28 <module>
    ai_msg: AIMessage(
        content='',
        additional_kwargs={
            'tool_calls': [
                {
                    'id': 'call_W5rxC9tNc6Si14S6zQCy40dQ',
                    'function': {
                        'arguments': '{"a":45,"b":12}',
                        'name': 'add',
                    },
                    'type': 'function',
                },
            ],
            'refusal': None,
        },
        response_metadata={
            'token_usage': {
                'completion_tokens': 17,
                'prompt_tokens': 99,
                'total_tokens': 116,
                'completion_tokens_details': None,
                'prompt_tokens_details': None,
            },
            'model_name': 'gpt-4',
            'system_fingerprint': 'fp_5603ee5e2e',
            'finish_reason': 'tool_calls',
            'logprobs': None,
            'content_filter_results': {},
        },
        id='run

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_W5rxC9tNc6Si14S6zQCy40dQ', 'function': {'arguments': '{"a":45,"b":12}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 99, 'total_tokens': 116, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_5603ee5e2e', 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-9b87d7d4-f2fc-49ba-8e27-113bde8f35b2-0', tool_calls=[{'name': 'add', 'args': {'a': 45, 'b': 12}, 'id': 'call_W5rxC9tNc6Si14S6zQCy40dQ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 99, 'output_tokens': 17, 'total_tokens': 116, 'input_token_details': {}, 'output_token_details': {}})

In [11]:
debug(llm_with_tools)

/tmp/ipykernel_5275/3318825891.py:1 <module>
    llm_with_tools: RunnableBinding(
        bound=AzureChatOpenAI(
            name='gpt4-turbo',
            client=<openai.resources.chat.completions.Completions object at 0x7f557c553f80>,
            async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f557c395b20>,
            root_client=<openai.lib.azure.AzureOpenAI object at 0x7f55b47a4740>,
            root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x7f557c553fb0>,
            model_name='gpt4-turbo',
            temperature=0.0,
            model_kwargs={},
            openai_api_key=SecretStr('**********'),
            disabled_params={
                'parallel_tool_calls': None,
            },
            azure_endpoint='https://mutualizedopenai.openai.azure.com',
            deployment_name='gpt4-turbo',
            openai_api_version='2023-05-15',
            openai_api_type='azure',
        ),
        kwargs={
            'tools': [
  

RunnableBinding(bound=AzureChatOpenAI(name='gpt4-turbo', client=<openai.resources.chat.completions.Completions object at 0x7f557c553f80>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f557c395b20>, root_client=<openai.lib.azure.AzureOpenAI object at 0x7f55b47a4740>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x7f557c553fb0>, model_name='gpt4-turbo', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), disabled_params={'parallel_tool_calls': None}, azure_endpoint='https://mutualizedopenai.openai.azure.com', deployment_name='gpt4-turbo', openai_api_version='2023-05-15', openai_api_type='azure'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': 'Addition 2 integer numbers a and b.\n\n    Args:\n        a: first int\n        b: second int', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config

In [ ]:
# Step 2 : Execute the function, and send result to LLM

In [12]:
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])

    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

r = llm_with_tools.invoke(messages)

debug(r)

'add' tool called
/tmp/ipykernel_5275/3057862763.py:11 <module>
    r: AIMessage(
        content='The sum of 45 and 12 is 57.',
        additional_kwargs={
            'refusal': None,
        },
        response_metadata={
            'token_usage': {
                'completion_tokens': 13,
                'prompt_tokens': 87,
                'total_tokens': 100,
                'completion_tokens_details': None,
                'prompt_tokens_details': None,
            },
            'model_name': 'gpt-4',
            'system_fingerprint': 'fp_5603ee5e2e',
            'finish_reason': 'stop',
            'logprobs': None,
            'content_filter_results': {},
        },
        id='run-96560d6e-cdc1-41e5-9fbb-55bd9a303523-0',
        usage_metadata={
            'input_tokens': 87,
            'output_tokens': 13,
            'total_tokens': 100,
            'input_token_details': {},
            'output_token_details': {},
        },
    ) (AIMessage)


AIMessage(content='The sum of 45 and 12 is 57.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 87, 'total_tokens': 100, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_5603ee5e2e', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-96560d6e-cdc1-41e5-9fbb-55bd9a303523-0', usage_metadata={'input_tokens': 87, 'output_tokens': 13, 'total_tokens': 100, 'input_token_details': {}, 'output_token_details': {}})

In [15]:
debug(messages)

/tmp/ipykernel_5275/4208883321.py:1 <module>
    messages: [
        AIMessage(
            content='',
            additional_kwargs={
                'tool_calls': [
                    {
                        'id': 'call_W5rxC9tNc6Si14S6zQCy40dQ',
                        'function': {
                            'arguments': '{"a":45,"b":12}',
                            'name': 'add',
                        },
                        'type': 'function',
                    },
                ],
                'refusal': None,
            },
            response_metadata={
                'token_usage': {
                    'completion_tokens': 17,
                    'prompt_tokens': 99,
                    'total_tokens': 116,
                    'completion_tokens_details': None,
                    'prompt_tokens_details': None,
                },
                'model_name': 'gpt-4',
                'system_fingerprint': 'fp_5603ee5e2e',
                'finish_reason': '

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_W5rxC9tNc6Si14S6zQCy40dQ', 'function': {'arguments': '{"a":45,"b":12}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 99, 'total_tokens': 116, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_5603ee5e2e', 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-9b87d7d4-f2fc-49ba-8e27-113bde8f35b2-0', tool_calls=[{'name': 'add', 'args': {'a': 45, 'b': 12}, 'id': 'call_W5rxC9tNc6Si14S6zQCy40dQ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 99, 'output_tokens': 17, 'total_tokens': 116, 'input_token_details': {}, 'output_token_details': {}}),
 ToolMessage(content='57', tool_call_id='call_W5rxC9tNc6Si14S6zQCy40dQ')]

###  Other method : Use a tool calling agent

In [14]:
import math

from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate

llm = get_llm("gpt_4_azure")
#llm = get_llm("llama32_3_ollama")

@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def exponentiate(x: float, y: float) -> float:
    """Calculate the power of a number. Return x**y (w to the power of y)"""
    print("exponentiate")
    return math.pow(x, y)


# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful Math Assistant. Please use the provided tool"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [add, exponentiate]
agent = create_tool_calling_agent(llm, tools, prompt)  # type: ignore
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)  # type: ignore

r = agent_executor.invoke({"input": "what is 12  + 100^3"})
debug(r)

2024-11-14 09:55:56.337 | INFO     | ai_core.llm:get_llm:409 - get LLM:'gpt_4_azure'




> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 12, 'b': 1000000}`


1000012
Invoking: `exponentiate` with `{'x': 100, 'y': 3}`


exponentiate
1000000.0The result of \( 12 + 100^3 \) is 1,000,012.

> Finished chain.
/tmp/ipykernel_5275/29369776.py:41 <module>
    r: {
        'input': 'what is 12  + 100^3',
        'output': 'The result of \\( 12 + 100^3 \\) is 1,000,012.',
    } (dict) len=2


{'input': 'what is 12  + 100^3',
 'output': 'The result of \\( 12 + 100^3 \\) is 1,000,012.'}

In [ ]:
## Call Eden.ai provided tools